In [26]:
from darkflow.net.build import TFNet
import cv2
import numpy as np

In [27]:
options = {"model": "cfg/yolo.cfg", "load": "bin/yolo.weights", "threshold": 0.1}
tfnet = TFNet(options)

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.04831194877624512s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!

In [28]:
imgcv = cv2.imread("./sample_img/sample_computer.jpg")
result = tfnet.return_predict(imgcv)
print(result)

[{'label': 'tvmonitor', 'confidence': 0.12422139, 'topleft': {'x': 323, 'y': 147}, 'bottomright': {'x': 409, 'y': 313}}, {'label': 'cup', 'confidence': 0.4583792, 'topleft': {'x': 318, 'y': 145}, 'bottomright': {'x': 423, 'y': 317}}, {'label': 'cup', 'confidence': 0.36555868, 'topleft': {'x': 374, 'y': 346}, 'bottomright': {'x': 421, 'y': 374}}, {'label': 'spoon', 'confidence': 0.1217906, 'topleft': {'x': 464, 'y': 259}, 'bottomright': {'x': 499, 'y': 357}}, {'label': 'bowl', 'confidence': 0.24720718, 'topleft': {'x': 419, 'y': 316}, 'bottomright': {'x': 484, 'y': 364}}, {'label': 'tvmonitor', 'confidence': 0.8816117, 'topleft': {'x': 157, 'y': 94}, 'bottomright': {'x': 345, 'y': 280}}, {'label': 'keyboard', 'confidence': 0.7986625, 'topleft': {'x': 123, 'y': 263}, 'bottomright': {'x': 333, 'y': 371}}, {'label': 'refrigerator', 'confidence': 0.48706755, 'topleft': {'x': 0, 'y': 19}, 'bottomright': {'x': 130, 'y': 351}}]


In [29]:
class_names = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
              'bus', 'car', 'cat', 'chair', 'cow', 'diningtable',
              'dog', 'horse', 'motorbike', 'person', 'pottedplant',
              'sheep', 'sofa', 'train', 'tvmonitor', 'keyboard', "book", "laptop"]

num_classes = len(class_names)
class_colors = []

In [30]:
for i in range(num_classes):
#     hue = 255 * i / num_classes
    hue = 64
    col = np.zeros((1, 1, 3)).astype("uint8")
    col[0][0][0] = hue
    col[0][0][1] = 128
    col[0][0][2] = 255
    cvcol = cv2.cvtColor(col, cv2.COLOR_HSV2BGR)
    col = (int(cvcol[0][0][0]), int(cvcol[0][0][1]), int(cvcol[0][0][2]))
    class_colors.append(col)

In [31]:
for item in result:
    tlx = item["topleft"]["x"]
    tly = item["topleft"]["y"]
    brx = item["bottomright"]["x"]
    bry = item["bottomright"]["y"]
    label = item["label"]
    conf = item["confidence"]
    
    if conf > 0.6:
        for class_name in class_names:
            if label == class_name:
                class_num = class_names.index(class_name)
                break
        
        cv2.rectangle(imgcv, (tlx, tly), (brx, bry), class_colors[class_num], 2)
        
        text = label + " " + ("%.2f" % conf)
        cv2.rectangle(imgcv, (tlx, tly - 15), (tlx + 150, tly + 5), class_colors[class_num], -1)
        cv2.putText(imgcv, text, (tlx, tly), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

In [32]:
cv2.imwrite("./sample_img/sample_computer2.jpg", imgcv)

True